In [0]:
select * from 
/*
update  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES set
       CPPROP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
       CPPROP_UPDATE_PROCESS_KEY = -999,
       CPPROP_CURRENT_FLAG = 'N', 
       CPPROP_VALID_TO = to_date('2025-09-05','yyyy-MM-dd')-1
 where CPPROP_CURRENT_FLAG = 'Y'
   and CPPROP_VALID_TO = to_date('30000101','yyyyMMdd')
   and (cpprop_key||'.'||cpprop_valid_from||'.'||ctp_key||'.'||case_start_date) in (select tech_rid from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_HUMANTASK_ADS_MAP_SCD_DIFF where tech_rid is not null)
   AND CPPTP_KEY in (select CPPTP_KEY from gap_catalog.ads_owner.CASE_PHASE_PROPERTY_TYPES CPPTP
           WHERE CPPTP.CPPTP_SOURCE_SYSTEM_ID = 'RDS'
                    AND CPPTP.CPPTP_SOURCE_SYS_ORIGIN = 'RDS_ANALYTICALCPPROPERTYTYPES'
                    AND CPPTP.CPPTP_VALID_TO= DATE'3000-01-01' 
   );
*/

describe table gap_catalog.ads_owner.CASE_PHASE_PROPERTIES;
describe table gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING;
describe table gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_MAIN;


 describe table gap_catalog.ads_owner.SMA_CASE_PHASE_PROPERTIES;
 describe table gap_catalog.ads_owner.CASES;

select count(1) from  gap_catalog.ads_owner.SMA_CASE_PHASE_PROPERTIES

change 
ctp_key

ALTER TABLE gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
ALTER COLUMN ctp_key TYPE BIGINT;

select * from temp_cpptp_keys;

ALTER TABLE gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_MAIN 
SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

-- Step 1: Add new column with desired datatype
ALTER TABLE gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_MAIN
ADD COLUMN CPPROP_KEY_NEW DECIMAL(38,0);

-- Step 2: Copy data from old column to new column
/*UPDATE gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_MAIN
SET CPPROP_KEY_NEW = CAST(CPPROP_KEY AS DECIMAL(38,0));*/

-- Step 3: Drop the old column
ALTER TABLE gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_MAIN
DROP COLUMN CPPROP_KEY;

-- Step 5: Rename new column to original name
ALTER TABLE gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_MAIN
RENAME COLUMN CPPROP_KEY_NEW TO CPPROP_KEY;

--------------------------------------------------

truncate table gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF;


INSERT INTO gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF
SELECT 
    'N' AS tech_del_flg, 
    CASE WHEN trg.CPPROP_KEY IS NULL THEN 'Y' ELSE 'N' END AS tech_new_rec,
    trg.rid AS tech_rid,
    src.CPPROP_KEY, 
    src.CPPROP_SOURCE_ID, 
    src.CPPROP_SOURCE_SYSTEM_ID, 
    src.CPPROP_SOURCE_SYS_ORIGIN, 
    src.CPPROP_PARENT_KEY, 
    src.CASEPH_KEY, 
    src.CPPTP_KEY, 
    src.CTP_KEY, 
    src.CASE_KEY, 
    src.CASE_START_DATE, 
    src.CPPROP_VALUE_TEXT, 
    src.CPPROP_VALUE_DATE, 
    src.CPPROP_VALUE_NUMBER, 
    src.CPPRV_KEY
FROM 
    (SELECT 
        CPPROP_KEY, 
        CPPROP_SOURCE_ID, 
        CPPROP_SOURCE_SYSTEM_ID, 
        CPPROP_SOURCE_SYS_ORIGIN, 
        CPPROP_PARENT_KEY, 
        CASEPH_KEY, 
        CPPTP_KEY, 
        CTP_KEY, 
        CASE_KEY, 
        CASE_START_DATE, 
        CPPROP_VALUE_TEXT, 
        CPPROP_VALUE_DATE, 
        CPPROP_VALUE_NUMBER, 
        CPPRV_KEY
     FROM gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING xc
         WHERE (CPPROP_SOURCE_SYSTEM_ID = 'SMA'
            AND CPPROP_SOURCE_SYS_ORIGIN = 'SMA_MONITOR_EVENTS'
            AND CTP_KEY = 1429
            AND CPPTP_KEY IN (
                SELECT DISTINCT CPPTP_KEY 
                FROM gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING
            ))
    ) src
LEFT JOIN
    (SELECT  
        CAST(cpprop_key AS STRING) || '.' || 
        CAST(cpprop_valid_from AS STRING) || '.' || 
        CAST(ctp_key AS STRING) || '.' || 
        CAST(case_start_date AS STRING) AS rid, 
        t.* 
     FROM  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES t
      join temp_cpptp_keys k on k.CPPTP_KEY = t.CPPTP_KEY
     WHERE CPPROP_CURRENT_FLAG = 'Y'
       AND CPPROP_VALID_TO = to_date('01013000','ddMMyyyy')
       AND CPPROP_SOURCE_SYSTEM_ID = 'SMA'
       AND CPPROP_SOURCE_SYS_ORIGIN = 'SMA_MONITOR_EVENTS'
       AND CTP_KEY = 1429
    ) trg 
ON trg.CPPROP_KEY = src.CPPROP_KEY
   AND trg.CPPROP_VALID_TO = to_date('30000101','yyyyMMdd') 
/*WHERE (
    NOT (src.CPPROP_SOURCE_ID <=> trg.CPPROP_SOURCE_ID) OR
    NOT (src.CPPROP_SOURCE_SYSTEM_ID <=> trg.CPPROP_SOURCE_SYSTEM_ID) OR
    NOT (src.CPPROP_SOURCE_SYS_ORIGIN <=> trg.CPPROP_SOURCE_SYS_ORIGIN) OR
    NOT (src.CPPROP_PARENT_KEY <=> trg.CPPROP_PARENT_KEY) OR
    NOT (src.CASEPH_KEY <=> trg.CASEPH_KEY) OR
    NOT (src.CPPTP_KEY <=> trg.CPPTP_KEY) OR
    NOT (src.CTP_KEY <=> trg.CTP_KEY) OR
    NOT (src.CASE_KEY <=> trg.CASE_KEY) OR
    NOT (src.CASE_START_DATE <=> trg.CASE_START_DATE) OR
    NOT (src.CPPROP_VALUE_TEXT <=> trg.CPPROP_VALUE_TEXT) OR
    NOT (src.CPPROP_VALUE_DATE <=> trg.CPPROP_VALUE_DATE) OR
    NOT (src.CPPROP_VALUE_NUMBER <=> trg.CPPROP_VALUE_NUMBER) OR
    NOT (src.CPPRV_KEY <=> trg.CPPRV_KEY) OR
    trg.CPPROP_KEY IS NULL 
    OR  trg.CPPROP_DELETED_FLAG = 'Y')
    */
;

;
-- Original 10 minut!
-- 1:58 -- bez decode + cast

CREATE OR REPLACE TEMP VIEW temp_cpptp_keys AS
SELECT DISTINCT CAST(CPPTP_KEY AS DECIMAL(38,0)) AS CPPTP_KEY
FROM gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING;

select * from temp_cpptp_keys;

 update  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES set
       CPPROP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
       CPPROP_UPDATE_PROCESS_KEY = -999,
       CPPROP_CURRENT_FLAG = 'N', 
       CPPROP_VALID_TO = to_date('20250911','yyyyMMdd')-1
 where CPPROP_CURRENT_FLAG = 'Y'
   and CPPTP_KEY IN (SELECT CPPTP_KEY FROM temp_cpptp_keys )
   and CPPROP_VALID_TO = to_date('30000101','yyyyMMdd')
   --and cpprop_key||'.'||cpprop_valid_from||'.'||ctp_key||'.'||case_start_date 
   and cpprop_key
   in (select cpprop_key from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF where tech_rid is not null)
   ;

MERGE INTO gap_catalog.ads_owner.CASE_PHASE_PROPERTIES AS trg
USING (
    SELECT DISTINCT d.tech_rid, k.CPPTP_KEY
    FROM gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF d
    CROSS JOIN temp_cpptp_keys k
    WHERE d.tech_rid IS NOT NULL
) AS src
ON trg.cpprop_key = src.tech_rid
   AND trg.CPPROP_CURRENT_FLAG = 'Y'
   AND trg.CPPROP_VALID_TO = to_date('30000101','yyyyMMdd')
   AND trg.CPPTP_KEY = src.CPPTP_KEY
WHEN MATCHED THEN UPDATE SET
    trg.CPPROP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
    trg.CPPROP_UPDATE_PROCESS_KEY = -999,
    trg.CPPROP_CURRENT_FLAG = 'N',
    trg.CPPROP_VALID_TO = to_date('20250911','yyyyMMdd') - 1;

    -- 3:16 minut bez conact
    -- 5:47 minut

select CPPTP_KEY,count(1)
from 
gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
group by CPPTP_KEY;


CREATE BLOOMFILTER INDEX
ON TABLE gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
FOR COLUMNS(
  CPPROP_KEY OPTIONS (fpp=0.1, numItems=20284747783)
);

gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
